## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
risk_factors_df = pd.read_csv("healthcare-dataset-stroke-data.csv")
risk_factors_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
risk_factors_df = risk_factors_df.drop(columns="id")

risk_factors_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
# Determine the number of unique values in each column.
risk_factors_df.nunique()

gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   418
smoking_status          4
stroke                  2
dtype: int64

In [4]:
#Drop all N/A values
risk_factors_df = risk_factors_df.dropna(how="any")

risk_factors_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
factor_dummies = pd.get_dummies(risk_factors_df[['gender','ever_married','work_type','Residence_type','smoking_status']])
#concat the dummies data onto the original dataframe
risk_factors_df = pd.concat([risk_factors_df, factor_dummies], axis=1)
#Drop original columns that have been converted to binary classification columns
risk_factors_df = risk_factors_df.drop(columns=['gender','ever_married','work_type','Residence_type','smoking_status'])

risk_factors_df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5,81.0,0,0,186.21,29.0,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0


In [6]:
# Split our preprocessed data into our features and target arrays
y = risk_factors_df.stroke.values
X = risk_factors_df.drop(columns="stroke").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

stroke_model = tf.keras.models.Sequential()

# First hidden layer
stroke_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=21))

# Second hidden layer
stroke_model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer - SIGMOID as the output layer, as we are classifying into a binary classification type
stroke_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
stroke_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                352       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
stroke_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = stroke_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


116/116 [==============================] - 0s 740us/step - loss: 0.4329 - accuracy: 0.8680
Epoch 2/100
116/116 [==============================] - 0s 749us/step - loss: 0.2049 - accuracy: 0.9573
Epoch 3/100
116/116 [==============================] - 0s 748us/step - loss: 0.1718 - accuracy: 0.9573
Epoch 4/100
116/116 [==============================] - 0s 731us/step - loss: 0.1605 - accuracy: 0.9573
Epoch 5/100
116/116 [==============================] - 0s 731us/step - loss: 0.1541 - accuracy: 0.9573
Epoch 6/100
116/116 [==============================] - 0s 731us/step - loss: 0.1496 - accuracy: 0.9571
Epoch 7/100
116/116 [==============================] - 0s 766us/step - loss: 0.1463 - accuracy: 0.9571
Epoch 8/100
116/116 [==============================] - 0s 757us/step - loss: 0.1442 - accuracy: 0.9573
Epoch 9/100
116/116 [==============================] - 0s 740us/step - loss: 0.1418 - accuracy: 0.9576
Epoch 10/100
116/116 [==============================] - 0s 705us/step - loss: 0.1402 

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = stroke_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1854 - accuracy: 0.9552 - 106ms/epoch - 3ms/step
Loss: 0.18544591963291168, Accuracy: 0.9552116990089417


In [12]:
# Export our model to HDF5 file
stroke_model.save("stroke_status_trained.h5")

c:\Users\Kronos\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
